In [1]:
using Pkg
Pkg.add(url = "https://github.com/MilesCranmer/SymbolicRegression.jl.git", rev = "support-extra-data")

    Updating git-repo `https://github.com/MilesCranmer/SymbolicRegression.jl.git`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [8254be44] ~ SymbolicRegression v0.22.4 ⇒ v0.22.2 `https://github.com/MilesCranmer/SymbolicRegression.jl.git#support-extra-data`
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [8254be44] ~ SymbolicRegression v0.22.4 ⇒ v0.22.2 `https://github.com/MilesCranmer/SymbolicRegression.jl.git#support-extra-data`
Precompiling project...
  ✓ SymbolicRegression
  1 dependency successfully precompiled in 42 seconds. 266 already precompiled.


In [1]:
using MLJ  # for fit/predict
using SymbolicRegression  # for SRRegressor
using Zygote  # For `enable_autodiff=true`
using SymbolicUtils
using NPZ
using Pandas

In [3]:
true_f(x) = x^3 / 3 - cos(x)
deriv_f(x) = x^2 + sin(x)

deriv_f (generic function with 1 method)

In [4]:
X = reshape(0.0:0.32:10.0, :, 1)
y = true_f.(X[:, 1])
∂y = deriv_f.(X[:, 1])

32-element Vector{Float64}:
  0.0
  0.41696656061611775
  1.006795441362392
  1.7407915683009982
  2.596415860289225
  3.5595736030415055
  4.626045473685325
  5.801915925084421
  7.102955436427127
  8.55301934966111
 10.181625856572422
 12.020959041455523
 14.102601257946091
  ⋮
 41.0765492048505
 45.58145539714299
 50.248209128707636
 55.050052009553035
 59.96730333407156
 64.98935824662338
 70.11576444366216
 75.35626809573589
 80.7298243269406
 86.26267271702045
 91.98567321877702
 97.93117297286523

In [5]:
function derivative_loss(tree, dataset::Dataset{T,L}, options, idx) where {T,L}
    # Select from the batch indices, if given
    X = idx === nothing ? dataset.X : view(dataset.X, :, idx)

    # Evaluate both f(x) and f'(x), where f is defined by `tree`
    ŷ, ∂ŷ, completed = eval_grad_tree_array(tree, X, options; variable=true)

    #println(size(dataset.extra.∂y))

    !completed && return L(Inf)

    y = idx === nothing ? dataset.y : view(dataset.y, idx)
    ∂y = idx === nothing ? dataset.extra.∂y : view(dataset.extra.∂y, idx)

    mse_deriv = sum(i -> (∂ŷ[i] - ∂y[i])^2, eachindex(∂y)) / length(∂y)
    mse_value = sum(i -> (ŷ[i] - y[i])^2, eachindex(y)) / length(y)

    return mse_value  + mse_deriv
end

derivative_loss (generic function with 1 method)

In [ ]:
model = SRRegressor(;
    binary_operators=[+, -, *],
    unary_operators=[cos],
    loss_function=derivative_loss,
    enable_autodiff=true,
    batching=true,
    batch_size=25,
    niterations=100,
    early_stop_condition=1e-6,
)
mach = machine(model, X, y, (; ∂y=∂y))
fit!(mach)

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc SymbolicRegression.SRRegressor` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{AbstractMatrix{Continuous}, AbstractVector{Continuous}, Table{AbstractVector{Continuous}}}
│ 
│ fit_data_scitype(model) = Union{Tuple{Union{Table{<:AbstractVector{<:Continuous}}, AbstractMatrix{<:Continuous}}, AbstractVector{<:Continuous}}, Tuple{Union{Table{<:AbstractVector{

In [ ]:
# load in the eta data

In [ ]:
datafile = npzread("toyproblem_sr_data.npz")

skip = 5
finish = 3000

X = datafile["theta"][1:skip:finish, :]
y = datafile["eta"][1:skip:finish, :]
∂y = datafile["jacobians"][1:skip:finish, :, :]

size(X), size(y), size(∂y)

In [2]:
pwd()

"/Users/lucas/repositories/degen_discovery/toy_problem"

In [ ]:
model = SRRegressor(;
    binary_operators=[+, *, ^],
    unary_operators=[log],
    constraints=[(^)=>(-1, 9)],
    nested_constraints=[(^) => [(^) => 0, log => 0],
                   log => [(^) =>  0, log => 0],
            #        exp => [log => 0]
        ],
    loss_function=derivative_loss,
    enable_autodiff=true,
    batching=false,
    #batch_size=100,
    niterations=100,
    parsimony=100,
)
mach = machine(model, X, y[:, 1], (; ∂y=∂y[:, 1, :]))
fit!(mach)

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc SymbolicRegression.SRRegressor` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{AbstractMatrix{Continuous}, AbstractVector{Continuous}, Unknown}
│ 
│ fit_data_scitype(model) = Union{Tuple{Union{Table{<:AbstractVector{<:Continuous}}, AbstractMatrix{<:Continuous}}, AbstractVector{<:Continuous}}, Tuple{Union{Table{<:AbstractVector{<:Continuous}}, AbstractMa

## loss for fitting $y=\partial y$
here we want to pass the derivatives as an extra and fit the candidate expression to that

In [ ]:
function fit_derivative_loss(tree, dataset::Dataset{T,L}, options, idx) where {T,L}
    # Column-major:
    X = idx === nothing ? dataset.X : view(dataset.X, :, idx)
    #∂y = idx === nothing ? dataset.y : view(dataset.y, idx)

    ŷ, ∂ŷ, completed = eval_grad_tree_array(tree, X, options; variable=true)

    !completed && return L(Inf)

    y = idx === nothing ? dataset.y : view(dataset.y, idx)
    ∂y = idx === nothing ? dataset.extra.∂y : view(dataset.extra.∂y, idx)
    
    # match the derivative only
    mse = sum(i -> (∂ŷ[i] - ∂y[i])^2, eachindex(∂y)) / length(∂y)
    
    return mse
end

In [ ]:
model = SRRegressor(;
    binary_operators=[+, *, ^],
    unary_operators=[log],
    constraints=[(^)=>(-1, 9)],
    nested_constraints=[(^) => [(^) => 0, log => 0],
                   log => [(^) =>  0, log => 0],
            #        exp => [log => 0]
        ],
    loss_function=fit_derivative_loss,
    enable_autodiff=true,
    #batching=false,
    batch_size=25,
    niterations=100,
    parsimony=100,
)
mach = machine(model, X, y[:, 1], (; ∂y=∂y[:, 1, :]))
fit!(mach)

# run script

In [ ]:
using MLJ  # for fit/predict
using SymbolicRegression  # for SRRegressor
using Zygote  # For `enable_autodiff=true`
using SymbolicUtils
using NPZ
using Pandas


function fit_derivative_loss(tree, dataset::Dataset{T,L}, options, idx) where {T,L}
    # Column-major:
    X = idx === nothing ? dataset.X : view(dataset.X, :, idx)
    #∂y = idx === nothing ? dataset.y : view(dataset.y, idx)

    ŷ, ∂ŷ, completed = eval_grad_tree_array(tree, X, options; variable=true)

    !completed && return L(Inf)

    y = idx === nothing ? dataset.y : view(dataset.y, idx)
    ∂y = idx === nothing ? dataset.extra.∂y : view(dataset.extra.∂y, idx)
    
    # match the derivative only
    mse = sum(i -> (∂ŷ[i] - ∂y[i])^2, eachindex(∂y)) / length(∂y)
    
    return mse
end


function derivative_loss(tree, dataset::Dataset{T,L}, options, idx) where {T,L}
    # Select from the batch indices, if given
    X = idx === nothing ? dataset.X : view(dataset.X, :, idx)

    # Evaluate both f(x) and f'(x), where f is defined by `tree`
    ŷ, ∂ŷ, completed = eval_grad_tree_array(tree, X, options; variable=true)

    #println(size(dataset.extra.∂y))

    !completed && return L(Inf)

    y = idx === nothing ? dataset.y : view(dataset.y, idx)
    ∂y = idx === nothing ? dataset.extra.∂y : view(dataset.extra.∂y, idx)

    mse_deriv = sum(i -> (∂ŷ[i] - ∂y[i])^2, eachindex(∂y)) / length(∂y)
    mse_value = sum(i -> (ŷ[i] - y[i])^2, eachindex(y)) / length(y)

    return  mse_deriv * mse_value
end




datafile = npzread("toyproblem_sr_data2.npz")

skip = 5
finish = 3000

X = datafile["theta"][1:skip:finish, :]
y = datafile["eta"][1:skip:finish, :]
∂y = datafile["jacobians"][1:skip:finish, :, :]

size(X), size(y), size(∂y)



# choose which variable to do

idx = 2

model = SRRegressor(;
    binary_operators=[+, *, ^],
    unary_operators=[log, sqrt],
    constraints=[(^)=>(-1, 9)],
    #nested_constraints=[(^) => [(^) => 0, log => 0],
    #               log => [(^) =>  0, log => 0],
                #    exp => [log => 0]
    #    ],
    loss_function=fit_derivative_loss,
    enable_autodiff=true,
    #batching=false,
    batch_size=30,
    niterations=100,
    #parsimony=100,
)
mach = machine(model, X, y[:, idx], (; ∂y=∂y[:, idx, :]))
fit!(mach)